In [77]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

serv = Service(ChromeDriverManager().install()) #update chrome version
driver = webdriver.Chrome(service=servico, options=options)

In [78]:
#open page
import os 

page_path = os.getcwd()
page = page_path + r"\index.html"

In [79]:
#import database
from selenium.webdriver import ActionChains
import pandas as pd

dataframe = pd.read_excel("Processos.xlsx")

for line in dataframe.index:
    driver.get(page)
    
    #find option / select option
    button = driver.find_element(By.CLASS_NAME, "dropdown-menu")
    
    #rover over element
    ActionChains(driver).move_to_element(button).perform()
    
    #get city info from excel
    city = dataframe.loc[line, "Cidade"]
    driver.find_element(By.PARTIAL_LINK_TEXT, city).click()
    
    #switching betwen tabs
    main_tab = driver.window_handles[0]
    index = 1 + line
    new_tab = driver.window_handles[index]
    driver.switch_to.window(new_tab)
    
    #fill info
    name = driver.find_element(By.ID, "nome").send_keys(dataframe.loc[line, "Nome"])
    lawyer = driver.find_element(By.ID, "advogado").send_keys(dataframe.loc[line, "Advogado"])
    number = driver.find_element(By.ID, "numero").send_keys(dataframe.loc[line, "Processo"])
    driver.find_element(By.CLASS_NAME, "registerbtn").click()
    
    
    #accept alert
    cont = 0
    while True:
        try:
            WebDriverWait(driver, 3).until(EC.alert_is_present(),
                                           'Timed out waiting for PA creation ' +
                                           'confirmation popup to appear.')
            alert = driver.switch_to.alert
            alert_text = alert.text
            print(alert_text)
            alert.accept()
            #print("Alert accepted!")
            cont += 1
        except TimeoutException:
            #print("No alert.")
            if cont == 2:
                break
    if "Nenhum processo encontrado" in alert_text:
        print("No process found.")
        dataframe.loc[line, "Status"] = "Not Found"
    else:
        print("Process found.")
        dataframe.loc[line, "Status"] = "Found"


Vamos buscar no nosso banco de dados, por favor aguarde. Pode durar alguns minutos.
Processo encontrado com sucesso! Deseja acessar?
Process found.
Vamos buscar no nosso banco de dados, por favor aguarde. Pode durar alguns minutos.
Processo encontrado com sucesso! Deseja acessar?
Process found.
Vamos buscar no nosso banco de dados, por favor aguarde. Pode durar alguns minutos.
Processo encontrado com sucesso! Deseja acessar?
Process found.
Vamos buscar no nosso banco de dados, por favor aguarde. Pode durar alguns minutos.
Processo encontrado com sucesso! Deseja acessar?
Process found.


In [82]:
driver.quit()

display(dataframe)
dataframe.to_excel("Update Processes.xlsx")

,Nome,Advogado,Processo,Cidade,Status
0,Lira,Alon Lawyer,PC6592,Distrito Federal,Found
1,João,Lawyer Alon,EB3792,Rio de Janeiro,Found
2,Amanda,Amanda mesmo,MM1043,Rio de Janeiro,Found
3,Carol,Amanda,PC5197,São Paulo,Found
